In [1]:
import pandas as pd
from scipy import stats

# 1. Wczytanie danych
df = pd.read_csv('dataset_cars.csv')

# Przygotowanie grup (kolumny x1, x2, x3, x4)
groups = [df['x1'], df['x2'], df['x3'], df['x4']]

# 2. Test normalności Shapiro-Wilka
print("--- Testy normalności ---")
for i, group in enumerate(groups, 1):
    stat, p = stats.shapiro(group)
    print(f"Marka x{i}: p-value = {p:.4f}")

# 3. Test homogeniczności wariancji Levene'a
stat_lev, p_lev = stats.levene(*groups)
print(f"\nTest Levene'a (homogeniczność): p-value = {p_lev:.4f}")

# 4. Jednoczynnikowa ANOVA
f_stat, p_anova = stats.f_oneway(*groups)
print(f"\n--- Wyniki ANOVA ---")
print(f"Statystyka F = {f_stat:.3f}")
print(f"p-value = {p_anova:.3f}")

# 5. Opcjonalnie: Test Kruskala-Wallisa (jeśli ANOVA nie jest 'najwłaściwsza' przez brak homogeniczności)
k_stat, p_k = stats.kruskal(*groups)
print(f"\nTest Kruskala-Wallisa: H = {k_stat:.3f}, p = {p_k:.3f}")

--- Testy normalności ---
Marka x1: p-value = 0.9769
Marka x2: p-value = 0.7254
Marka x3: p-value = 0.4492
Marka x4: p-value = 0.2792

Test Levene'a (homogeniczność): p-value = 0.0000

--- Wyniki ANOVA ---
Statystyka F = 7.717
p-value = 0.000

Test Kruskala-Wallisa: H = 21.422, p = 0.000


In [2]:
# II PODEJŚCIE 
import pandas as pd
from scipy import stats
import numpy as np

# 1. Wczytanie danych
df = pd.read_csv('dataset_cars (1).csv')
groups = [df['x1'].dropna(), df['x2'].dropna(), df['x3'].dropna(), df['x4'].dropna()]

# --- OPCJA 1: TEST KRUSKALA-WALLISA (Statystyka H) ---
# Często uznawany za "najbezpieczniejszy" zamiennik ANOVA
h_stat, p_kruskal = stats.kruskal(*groups)
print(f"Test Kruskala-Wallisa (H): {h_stat:.3f}")
print(f"p-value Kruskal: {p_kruskal:.3f}")

# --- OPCJA 2: ANOVA WELCHA (Statystyka F Welcha) ---
# Bardziej precyzyjny zamiennik ANOVA przy braku homogeniczności wariancji
def welch_anova(groups):
    k = len(groups)
    n = np.array([len(g) for g in groups])
    m = np.array([np.mean(g) for g in groups])
    v = np.array([np.var(g, ddof=1) for g in groups])
    w = n / v
    W = np.sum(w)
    m_w = np.sum(w * m) / W
    num = np.sum(w * (m - m_w)**2) / (k - 1)
    lambdas = (1 - w / W)**2 / (n - 1)
    den = 1 + 2 * (k - 2) / (k**2 - 1) * np.sum(lambdas)
    f_welch = num / den
    # df1 = k - 1, df2 = (k**2 - 1) / (3 * np.sum(lambdas))
    return f_welch

f_welch = welch_anova(groups)
print(f"ANOVA Welcha (F): {f_welch:.3f}")

Test Kruskala-Wallisa (H): 35.427
p-value Kruskal: 0.000
ANOVA Welcha (F): 21.152
